In [37]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from dataset_train import Dataset
from torch.utils.data import DataLoader

import os
import cv2
import numpy as np

from time import time
from visdom import Visdom
import copy

# Define important global variables 
# Change 'pretrained' variable to decide to get the pretrained weight or not
# Change 'feature_extract' variable to decide to keep all layers to update or freeze a part of the model
num_classes = 7
batch_size = 32
feature_extract = False
pretrained = False
num_epochs = 1000

# Input size of ResNet-18
input_size = 224

# Use Visdom to visualise the training progress
viz = Visdom()
step_list = [0]
win = viz.line(X=np.array([0]), Y=np.array([1.0]), opts=dict(title='loss'))


Setting up a new session...


In [38]:
# This function is used to tell the optimizer not to update params in the freeze layer 
# Only use in freeze mode
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [39]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 1000.0

    count = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0

        count += 1
        # Iterate over data.
        for step, (inputs, labels) in enumerate(dataloaders):
            inputs = inputs.cuda()
            labels = labels.cuda()
            

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward pass and calculate loss
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            # Back prop and update params
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

            if step % 1 == 0:
                step_list.append(step_list[-1] + 1)
                viz.line(X=np.array([step_list[-1]]), Y=np.array([loss.item()]), win=win, update='append')
                
            if step % 100 == 0:    
                print('     step:{}, loss:{:.3f}, time:{:.3f} min'
                    .format(step, loss.item(), (time() - since) / 60))

        # deep copy the model
        if loss.item() < best_acc:
            count = 0
            best_acc = loss.item()
            best_model_wts = copy.deepcopy(model.state_dict())
            
            if feature_extract and pretrained:
                torch.save(model.state_dict(), './model/netFE{}-{:.3f}.pth'.format(epoch, loss))
            elif (not feature_extract) and pretrained:
                torch.save(model.state_dict(), './model/netFinetune{}-{:.3f}.pth'.format(epoch, loss))
            elif (not feature_extract) and (not pretrained):
                torch.save(model.state_dict(), './model/netScratch{}-{:.3f}.pth'.format(epoch, loss))

        if count == 10:
            break
        print()

    time_elapsed = time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [40]:
BASE_DIR = '/home/jun/Github/BoVW/dataset/'
impath = os.listdir(BASE_DIR + 'train')

# Load data
train_ds = Dataset(BASE_DIR)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Init model and customize from the pretrained version in Pytorch
model = models.resnet18(pretrained=pretrained).cuda()
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,num_classes)

model.cuda()
# model =  torch.nn.DataParallel(model).cuda()

loss_fn = torch.nn.CrossEntropyLoss()

# Choose params to update 
params_to_update = model.parameters()
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            
optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

['sea', 'green', 'city', 'house_building', 'face', 'house_indoor', 'office']


In [41]:
torch.cuda.empty_cache()

model_ft = train_model(model, train_dl, loss_fn, optimizer, num_epochs=num_epochs)

Epoch 0/999
----------
     step:0, loss:1.952, time:0.006 min

Epoch 1/999
----------
     step:0, loss:1.471, time:0.214 min

Epoch 2/999
----------
     step:0, loss:1.130, time:0.401 min

Epoch 3/999
----------
     step:0, loss:0.684, time:0.592 min

Epoch 4/999
----------
     step:0, loss:0.664, time:0.785 min

Epoch 5/999
----------
     step:0, loss:0.511, time:0.977 min

Epoch 6/999
----------
     step:0, loss:0.404, time:1.169 min

Epoch 7/999
----------
     step:0, loss:0.364, time:1.352 min

Epoch 8/999
----------
     step:0, loss:0.342, time:1.550 min

Epoch 9/999
----------
     step:0, loss:0.405, time:1.745 min

Epoch 10/999
----------
     step:0, loss:0.248, time:1.936 min

Epoch 11/999
----------
     step:0, loss:0.234, time:2.124 min

Epoch 12/999
----------
     step:0, loss:0.312, time:2.311 min

Epoch 13/999
----------
     step:0, loss:0.256, time:2.504 min

Epoch 14/999
----------
     step:0, loss:0.319, time:2.689 min

Epoch 15/999
----------
     step:0